In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from tensorflow import keras

In [2]:
import comm
import time
class MFCEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(2)
        # Temperature array
        self.observation_space = Box(low=np.array([0, 0, 0, 0]), high=np.array([100, 100, 100, 100]), shape=(4,), dtype=np.float32)
        # Set start temp
        self.state = np.array(comm.get_data())
        comm.turn_off(0)
        # Time
        self.length = 2
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        last_state = self.state
        if action == 0:
            comm.turn_off(60)
        else:
            comm.turn_on(60)
        time.sleep(60)
        self.state = np.array(comm.get_data())
        
        # Calculate reward
        if self.state[0] > last_state[0]:
            reward = 1 
        else: 
            reward = -1 
        
        self.length -= 1
        done = False
        if self.length == 0:
            done = True
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = np.array(comm.get_data())
        comm.turn_off(0)
        time.sleep(2)
        self.length = 5
        return self.state

Server Created


In [3]:
env = MFCEnv()

c:\Users\tanay\anaconda3\envs\mfc\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Connection from ('192.168.4.150', 45768) has been established fr!


In [4]:
env.observation_space.sample()

array([82.06543 , 70.19154 , 29.2579  , 55.956287], dtype=float32)

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM
from tensorflow.keras.optimizers import Adam

In [6]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
def build_model(states, actions):
    model = Sequential()    
    model.add(LSTM(24, activation='relu', input_shape=(1,4)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))

    return model

In [8]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

model = build_model(states, actions)

In [9]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=9, target_model_update=1e-2)
    return dqn

In [10]:
from tensorflow.keras.optimizers import Adam
print(actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=60*9, visualize=False, verbose=1, log_interval=60*9)

2
Training for 540 steps ...
Connection from ('192.168.4.150', 45772) has been established fr!
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Sending message...
Message sent!
Connection from ('192.168.4.150', 45780) has been established fr!
  2/540 [..............................] - ETA: 9:00:36 - reward: 0.0000e+00Sending message...
Message sent!
Connection from ('192.168.4.150', 45788) has been established fr!
  3/540 [..............................] - ETA: 8:59:20 - reward: -0.3333   

In [ ]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((socket.gethostname(), 5979))
while True:
    msg = s.recv(8)
    print(msg.decode("utf-8"))